# FUNCION PARA INICIAR EL DRIVER DE SELENIUM

### Instalación de dependencias
En esta celda, actualizamos las dependencias del sistema y luego instalamos las librerías necesarias para trabajar con Selenium:  
- **Selenium**: para la automatización de navegadores web.  
- **webdriver-manager**: para gestionar automáticamente los controladores del navegador (Chrome, en este caso).

In [1]:
# 📦 Instalar dependencias
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,725 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,944 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 http://archive.ubuntu.com/u

In [2]:
!pip install -q selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 35.5 MB/s eta 0:00:00


### Configuración del entorno y opciones para Selenium
En esta celda:
1. Importamos las librerías necesarias de Selenium y otras utilidades.
2. Configuramos opciones del navegador Chrome:
   - **--headless**: ejecuta Chrome sin interfaz gráfica.
   - **--no-sandbox**: evita restricciones de seguridad en entornos virtualizados.
   - **--disable-dev-shm-usage**: mejora la gestión de memoria compartida en Docker.
3. Especificamos la ruta al controlador de Chrome.

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración más detallada de las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Mobile Safari/537.36')  # Establecer el User-Agent

In [4]:
def iniciar_driver():
    # ✅ Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')
    # # Inicializar el webdriver
    driver = webdriver.Chrome(options=chrome_options)
    return driver

# OBTENER TEXTO DEL FORO "REVIEWS" DEL JUEGO

In [5]:
driver= iniciar_driver()
foros = {} # Diccionario donde se almacenan los titulos de cada articulo del foro y el texto respectivo
url = "https://boardgamegeek.com/boardgame/295947/cascadia/forums/63"  # FORO DE REVIEWS (RESEÑAS)
driver.get(url)

# Esperar hasta que los elementos estén presentes
WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.summary-item-title")))

# Obtenemos los titulos de las reseñas del foro REVIEWS del juego
titulos_foros = driver.find_elements(By.CSS_SELECTOR, "div.summary-item-title")

# Iteramos sobre cada título de foro para obtener el enlace y el texto
for titulo in titulos_foros:
    enlaces = titulo.find_elements(By.TAG_NAME, "a")  # Encontrar todos los elementos "a" dentro del titulo

    if len(enlaces) >= 2:  # Verificar si hay al menos 2 elementos "a"
        enlace = enlaces[1]  # Obtener el segundo elemento "a"
        texto_foro = enlace.text.strip()  # Obtenemos el texto del enlace
        url_foro = enlace.get_attribute("ng-href")  # Obtenemos la URL del enlace
        #print(texto_foro) # Vemos el titulo del articulo
        #print(url_foro)   # Vemos el link del articulo

        # Navegamos al foro indicado por la URL anterior
        driver= iniciar_driver()
        # Construimos la URL completaL
        base_url = "https://boardgamegeek.com"
        full_url_foro = base_url + url_foro
        driver.get(full_url_foro)

        # Esperamos a que se cargue la pagina
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))

        # Scroll hasta la parte inferior de la pagina
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Esperar unos segundos para permitir que se cargue el contenido dinamico de la pagina
        time.sleep(5)

        # Obtener trodos los elementos con "gg-article" (que contienen los post de cada articulo)
        articulos = driver.find_elements(By.XPATH, "//*[@id='mainbody']/div/div/gg-thread-container/gg-thread/gg-articles/gg-articles-ui/section/gg-article/gg-post/article/div")

        texto_reseña = ""

        # Itera a través de cada artículo y agrega el texto a texto_reseña
        for articulo in articulos:
            texto_reseña += articulo.text + "\n"

        # Vemos la reseña
        print(f"---Reseña de '{texto_foro}")
        print(texto_reseña.replace('\n', ''))

        # Almacenamos la reseña en el diccionario
        foros[texto_foro] = texto_reseña

driver.quit()

---Reseña de 'Cascadia in 2025
It’s rare to find a game that does exactly what it promises—and Cascadia nails it. From the moment you see that cover art whispering “harmony with nature,” you know you’re in for a calm, satisfying experience.When you sit down to play, you find a serene puzzle… until that perfect combo of salmon tokens lands on the board. You know it isn’t actually the optimal play for your wife, but somehow she snags that salmon first and you realize even indirect competition can sting. Ten seconds later the frustation disappears as you realize there’s always another tile waiting. The game is smart enough to steer you clear of truly dead-end draws, so you never feel hopelessly stuck.Components get top marks from the tarot-sized cards, the high quality habitat tiles to the chunky animal tokens. My only nitpick? The insert is terrible. I still can’t figure out the “correct” way to pack it all away—strangely ironic for a game where everything else fits so perfectly.The game

In [8]:
# SCRIPT PARA GUARDAR LAS RESEÑAS DEL FORO "REVIEWS" DEL JUEGO EN ARCHIVO .TXT
import os

from google.colab import drive
drive.mount('/content/drive/')

# Nombre del archivo para guardar los enlaces
nombre_archivo = 'foro_reviews_cascadia.txt'

# # Acceso a la carpeta compartida en MyDrive
directorio_guardado = "/content/drive/MyDrive/Facultad/7mo Cuatrimestre - Marzo 2025/NLP/Tp 2/datos/información"
os.makedirs(directorio_guardado, exist_ok=True)

# Construir la ruta completa del archivo
ruta_completa = os.path.join(directorio_guardado, nombre_archivo)

# Abrir el archivo en modo escritura
with open(ruta_completa, 'w', encoding='utf-8') as archivo:
    for clave, valor in foros.items():
        archivo.write("===RESEÑA===\n")
        archivo.write(f"Título: {clave}\n")
        archivo.write(valor.strip() + "\n\n")


print(f"Foro 'Reviews' guardado en {ruta_completa}")

Mounted at /content/drive/
Foro 'Reviews' guardado en /content/drive/MyDrive/Facultad/7mo Cuatrimestre - Marzo 2025/NLP/Tp 2/datos/información/foro_reviews_cascadia.txt


In [7]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful